# パッケージインストール

In [ ]:
!pip install -q -U "vllm[audio]"
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q --upgrade "mistral-common[audio]"

# モデル読み込み

In [ ]:
import torch
from transformers import VoxtralForConditionalGeneration, AutoProcessor

device = "cuda"
repo_id = "mistralai/Voxtral-Mini-3B-2507"

processor = AutoProcessor.from_pretrained(repo_id)
model = VoxtralForConditionalGeneration.from_pretrained(
    repo_id,
    torch_dtype=torch.bfloat16,
    device_map=device
)

# 単一音声 + テキスト

In [3]:
!wget https://huggingface.co/datasets/hf-internal-testing/dummy-audio-samples/resolve/main/winning_call.mp3 -q

In [4]:
%%time
# 入力準備
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "audio",
                "path": "winning_call.mp3",  # 25秒のmp3
            },
            {"type": "text", "text": "どのスポーツが言及されていますか？"},
        ],
    }
]

inputs = processor.apply_chat_template(conversation)
inputs = inputs.to(device, dtype=torch.bfloat16)

CPU times: user 689 ms, sys: 29.6 ms, total: 719 ms
Wall time: 684 ms


In [5]:
%%time
# 推論
outputs = model.generate(**inputs, max_new_tokens=500)
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

CPU times: user 3.09 s, sys: 436 ms, total: 3.52 s
Wall time: 3.49 s


In [6]:
print(decoded_outputs[0])

この音声は野球（Baseball）についての試合の様子を伝えています。


# 複数音声 + テキスト

In [7]:
!wget https://huggingface.co/datasets/hf-internal-testing/dummy-audio-samples/resolve/main/mary_had_lamb.mp3 -q

In [8]:
%%time
# 入力準備
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "audio",
                "path": "mary_had_lamb.mp3",  # 15秒のmp3
            },
            {
                "type": "audio",
                "path": "winning_call.mp3",  # 25秒のmp3
            },
            {"type": "text", "text": "どのスポーツと、どの童謡が言及されていますか？」"},
        ],
    }
]

inputs = processor.apply_chat_template(conversation)
inputs = inputs.to(device, dtype=torch.bfloat16)

CPU times: user 829 ms, sys: 16.2 ms, total: 845 ms
Wall time: 787 ms


In [9]:
%%time
# 推論
outputs = model.generate(**inputs, max_new_tokens=500)
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

CPU times: user 5.18 s, sys: 423 ms, total: 5.6 s
Wall time: 5.55 s


In [10]:
print(decoded_outputs[0])

この音声は野球と童謡の「Mary Had a Little Lamb」が言及されています。


# 書き起こし

In [11]:
!wget https://github.com/Kazuhito00/Voxtral-Mini-3B-Colaboratory-Sample/raw/refs/heads/main/maou_14_shining_star.mp3 -q

In [12]:
%%time
# 入力準備
inputs = processor.apply_transcrition_request(
    language="ja",
    audio="maou_14_shining_star.mp3",  # 4分36秒の曲
    model_id=repo_id
)
inputs = inputs.to(device, dtype=torch.bfloat16)

CPU times: user 5.86 s, sys: 141 ms, total: 6 s
Wall time: 5.75 s


In [15]:
%%time
# 推論
outputs = model.generate(**inputs, max_new_tokens=500)
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

CPU times: user 1min 20s, sys: 4.24 s, total: 1min 24s
Wall time: 1min 24s


In [16]:
import textwrap

print(textwrap.fill(decoded_outputs[0], width=50))

ただ風に揺れて、何も考えずに、ただ雲を眺めて過ごすのもいいよね。誰しも何かしらの使命を持っている。た
だそのだけのことを悩むのはもうやめさ。さざなみの音に癒されて行く。奇跡を運ぶ風の目、時を閉じ込めて。
シャイニングスタを踏めば、夢に眠る魔法が手のひらに流れそそぐ新しい世界へ。I'll a dream
for my sensation、果てしなき道の向こうで、まぶたの裏に映る一滴の光、ときめきを感じて
。あなたが触れるすべて、幸せであるように、今生きる喜びを忘れてしまわないように。月の光を隠す雲は揺れ
、自由の翼は今も大空を駆ける。シャイニングスタを踏めば、無限のイマジネーション、魔法が使われるような
世界が広がる。I'll a dream for my sensation、今もフリッツの思いよ、切な
さやときめきが心の真ん中で、熱いメロディーになる。シャイニングスタを踏めば、無限のイマジネーション、
魔法が使われるような世界が広がる。I'll a dream for my sensation、今もフ
リッツの思いよ、切なさやときめきが輝き始める。シャイニングスタを踏めば、夢に眠る魔法が手のひらに流れ
そそぐ新しい世界へ。I'll a dream for my sensation、果てしなき道の向こう
で、まぶたの裏に映る一滴の光、ときめきを感じて。ララララララララララララララララララララララララララ
ララララララララララララララララララララララララララララララララララララララララララララララララララ
ラララ
